In [ ]:
! pip install langchain unstructured[all-docs] pydantic lxml openai chromadb tiktoken

## Data Loading

### Partition PDF tables, text, and images

* Use [Unstructured](https://unstructured-io.github.io/unstructured/) to partition elements

In [3]:
!pip install pytesseract

In [ ]:
!pip install torchvision

^C


In [1]:
import torchvision

In [1]:
from typing import Any
import os
from unstructured.partition.pdf import partition_pdf
import pytesseract
import os

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

input_path = os.getcwd()
output_path = os.path.join(os.getcwd(), "output")

# Get elements
raw_pdf_elements = partition_pdf(
    filename=os.path.join(input_path, "test.pdf"),
    extract_images_in_pdf=True,
    infer_table_structure=True,
    chunking_strategy="by_title",
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    image_output_dir_path=output_path
)

In [3]:
import base64

text_elements = []
table_elements = []
image_elements = []

# Function to encode images
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

for element in raw_pdf_elements:
    if 'CompositeElement' in str(type(element)):
        text_elements.append(element)
    elif 'Table' in str(type(element)):
        table_elements.append(element)

table_elements = [i.text for i in table_elements]
text_elements = [i.text for i in text_elements]

# Tables
print(len(table_elements))

# Text
print(len(text_elements))

3
26


In [4]:
output_path = r"./figures"
for image_file in os.listdir(output_path):
    if image_file.endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(output_path, image_file)
        image_elements.append(image_path)
print(len(image_elements))

22


In [6]:
# Import necessary packages and set API key
from aift.multimodal import textqa
from aift import setting

# Set the API key
setting.set_api_key('hChDEu87Ln2VThG8HVqEweoeVoVSGHgJ')

# Function for text summaries
def summarize_text(text_element):
    session_id = 'YOUR_SESSION'
    prompt = f"Summarize the following text:\n\n{text_element}\n\nSummary:"
    response = textqa.chat(prompt, sessionid=session_id, temperature=0.2, return_json=False)
    return response

# Function for table summaries
def summarize_table(table_element):
    session_id = 'YOUR_SESSION'
    prompt = f"Summarize the following table:\n\n{table_element}\n\nSummary:"
    response = textqa.chat(prompt, sessionid=session_id, temperature=0.2, return_json=False)
    return response

# Example usage
text_summary = summarize_text("AI เป็นสิ่งที่แวดวงธุรกิจ และอุตสาหกรรมให้ความสนใจกันเป็นอย่างมาก แม้แต่คนทั่วไปก็อาจเคยได้ยินคำว่า AI กันมานาน บางคนรู้จัก AI ผ่านภาพยนตร์ ซีรีส์ หรือจากเกมต่างๆ ซึ่งจริงๆ แล้ว AI เป็นส่วนหนึ่งของชีวิตประจำวันของเรามากกว่าที่คิด แต่หลายคนอาจไม่รู้ว่าจริงๆ แล้ว AI คืออะไร ระบบ AI มีหลักการทำงานอย่างไร AI มีกี่ประเภทกันแน่ในปัจจุบัน ทำไมถึงมีความสำคัญต่อธุรกิจ และอุตสาหกรรมนัก หรือที่หลายคนสงสัยว่า AI สามารถคิด และมีความรู้สึกจริงไหม ซึ่งในบทความนี้ได้นำความรู้ต่างๆ เกี่ยวกับเทคโนโลยี AI ว่าคืออะไร AI เข้ามามีบทบาทในการช่วยให้การทำงานในแวดวงอุตสาหกรรมสะดวกสบายอย่างไร เพื่อให้เข้าใจความหมายของ AI และรู้จักการทำงานของ AI มากขึ้น")
table_summary = summarize_table("Column1 | Column2\n------|------\nData1 | Data2")
print("Text Summary:", text_summary)
print("Table Summary:", table_summary)


Text Summary: AI เป็นสิ่งที่แวดวงธุรกิจ และอุตสาหกรรมให้ความสนใจกันเป็นอย่างมาก แม้แต่คนทั่วไปก็อาจเคยได้ยินคำว่า AI กันมานาน บางคนรู้จัก AI ผ่านภาพยนตร์ ซีรีส์ หรือจากเกมต่างๆ ซึ่งจริงๆ แล้ว AI เป็นส่วนหนึ่งของชีวิตประจำวันของเรามากกว่าที่คิด แต่หลายคนอาจไม่รู้ว่าจริงๆ แล้ว AI คืออะไร ระบบ AI มีหลักการทำงานอย่างไร AI มีกี่ประเภทกันแน่ในปัจจุบัน ทำไมถึงมีความสำคัญต่อธุรกิจ และอุตสาหกรรมนัก หรือที่หลายคนสงสัยว่า AI สามารถคิด และมีความรู้สึกจริงไหม ซึ่งในบทความนี้ได้นำความรู้ต่างๆ เกี่ยวกับเทคโนโลยี AI ว่าคืออะไร AI เข้ามามีบทบาทในการช่วยให้การทำงานในแวดวงอุตสาหกรรมสะดวกสบายอย่างไร เพื่อให้เข้าใจความหมายของ AI และรู้จักการทำงานของ AI มากขึ้น
Table Summary: The table contains two columns, Column1 and Column2, with two rows of data. The first row contains the data "Data1" in Column1 and "Data2" in Column2. The second row contains the data "Data1" in Column1 and "Data2" in Column2.


In [7]:
# import package and set API_KEY
from aift.multimodal import vqa
from aift import setting


vqa.generate(r'C:\Users\Admin\Desktop\LLM\Multimodal-RAG-With-OpenAI\large-language-models-llms-process.jpg', 'รูปนี้คืออะไร')
# output
# {'filename': '/path/to/your/image_file/car-demo.png',
#  'content_type': 'image/jpeg',
#  'query': 'รูปนี้คืออะไร',
#  'content': 'รถยนต์คันสีแดงจอดอยู่ข้างกับกำแพงสีขาวมีลายเส้นสีน้ำตาล',
#  'execution_time': '1.29'}

{'filename': 'large-language-models-llms-process.jpg',
 'content_type': 'image/jpeg',
 'query': 'รูปนี้คืออะไร',
 'content': '',
 'execution_time': '0.48'}

In [20]:
# Import necessary packages and set API key
from aift.multimodal import vqa
from aift import setting

# Function for image summaries
def summarize_image(image_path):
    query = "Describe the contents of this image."
    response = vqa.generate(image_path, query)
    print(response)
    description = response.get('content', 'No description available')
    execution_time = response.get('execution_time', 'N/A')
    return description

# Example usage
image_path = r"C:\Users\Admin\Desktop\LLM\Multimodal-RAG-With-OpenAI\figures\figure-3-1.jpg"  # Replace with actual image path
image_description= summarize_image(image_path)
print("Image Description:", image_description)



{'filename': 'figure-3-1.jpg', 'content_type': 'image/jpeg', 'query': 'Describe the contents of this image.', 'content': '', 'execution_time': '0.42'}
Image Description: 


In [21]:
# Processing table elements with feedback and sleep
table_summaries = []
for i, te in enumerate(table_elements[0:2]):
    summary = summarize_table(te)
    table_summaries.append(summary)
    print(f"{i + 1}th element of tables processed.")

1th element of tables processed.
2th element of tables processed.


In [22]:
# Processing text elements with feedback and sleep
text_summaries = []
for i, te in enumerate(text_elements[0:2]):
    summary = summarize_text(te)
    text_summaries.append(summary)
    print(f"{i + 1}th element of texts processed.")

1th element of texts processed.
2th element of texts processed.


In [23]:
image_elements

['./figures\\figure-14-5.jpg',
 './figures\\figure-15-6.jpg',
 './figures\\figure-18-10.jpg',
 './figures\\figure-18-11.jpg',
 './figures\\figure-18-12.jpg',
 './figures\\figure-18-13.jpg',
 './figures\\figure-18-14.jpg',
 './figures\\figure-18-15.jpg',
 './figures\\figure-18-16.jpg',
 './figures\\figure-18-17.jpg',
 './figures\\figure-18-7.jpg',
 './figures\\figure-18-8.jpg',
 './figures\\figure-18-9.jpg',
 './figures\\figure-19-18.jpg',
 './figures\\figure-19-19.jpg',
 './figures\\figure-19-20.jpg',
 './figures\\figure-19-21.jpg',
 './figures\\figure-19-22.jpg',
 './figures\\figure-3-1.jpg',
 './figures\\figure-4-2.jpg',
 './figures\\figure-7-3.jpg',
 './figures\\figure-8-4.jpg']

In [24]:
# Processing image elements with feedback and sleep
image_summaries = []
for i, ie in enumerate(image_elements[0:2]):
    summary = summarize_image(ie)
    image_summaries.append(summary)
    print(f"{i + 1}th element of images processed.")

{'filename': './figures\\figure-14-5.jpg', 'content_type': 'image/jpeg', 'query': 'Describe the contents of this image.', 'content': '', 'execution_time': '0.43'}
1th element of images processed.
{'filename': './figures\\figure-15-6.jpg', 'content_type': 'image/jpeg', 'query': 'Describe the contents of this image.', 'content': '', 'execution_time': '0.37'}
2th element of images processed.


In [25]:
image_summaries

['', '']

## Multi-vector retriever

Use [multi-vector-retriever](https://python.langchain.com/docs/modules/data_connection/retrievers/multi_vector#summary).

Summaries are used to retrieve raw tables and / or raw chunks of text.

### Add to vectorstore

Use [Multi Vector Retriever](https://python.langchain.com/docs/modules/data_connection/retrievers/multi_vector#summary) with summaries.

In [ ]:
!pip install langchain-qdrant

In [26]:
from sentence_transformers import SentenceTransformer

class CustomEmbeddings:
    def __init__(self, model_name="mrp/simcse-model-m-bert-thai-cased"):
        """
        Initialize the embedding model using SentenceTransformer.
        :param model_name: Name of the pre-trained model
        """
        self.model = SentenceTransformer(model_name)

    def embed_query(self, text):
        """
        Generate embeddings for a single query.
        :param text: Input text to embed
        :return: Embedding vector as a Python list
        """
        embedding = self.model.encode([text])
        return embedding[0].tolist()  # Convert NumPy array to list

    async def aembed_query(self, text):
        """
        Asynchronous version of `embed_query`.
        :param text: Input text to embed
        :return: Embedding vector as a Python list
        """
        return self.embed_query(text)

    def embed_documents(self, texts):
        """
        Generate embeddings for multiple documents.
        :param texts: List of input texts to embed
        :return: List of embedding vectors as Python lists
        """
        embeddings = self.model.encode(texts)
        return [embedding.tolist() for embedding in embeddings]

    async def aembed_documents(self, texts):
        """
        Asynchronous version of `embed_documents`.
        :param texts: List of input texts to embed
        :return: List of embedding vectors as Python lists
        """
        return self.embed_documents(texts)


In [27]:
import uuid

from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.schema.document import Document
from langchain.storage import InMemoryStore
from langchain.vectorstores import Chroma

# Initialize the vector store and storage layer
vectorstore = Chroma(collection_name="summaries", embedding_function=CustomEmbeddings(model_name="mrp/simcse-model-m-bert-thai-cased"))
store = InMemoryStore()
id_key = "doc_id"

# Initialize the retriever
retriever = MultiVectorRetriever(vectorstore=vectorstore, docstore=store, id_key=id_key)

# Function to add documents to the retriever
def add_documents_to_retriever(summaries, original_contents):
    doc_ids = [str(uuid.uuid4()) for _ in summaries]
    summary_docs = [
        Document(page_content=s, metadata={id_key: doc_ids[i]})
        for i, s in enumerate(summaries)
    ]
    retriever.vectorstore.add_documents(summary_docs)
    retriever.docstore.mset(list(zip(doc_ids, original_contents)))


d:\anaconda\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [28]:
# Add text summaries
add_documents_to_retriever(text_summaries, text_elements)

# Add table summaries
add_documents_to_retriever(table_summaries, table_elements)

# Add image summaries
add_documents_to_retriever(image_summaries, image_summaries) # hopefully real images soon

# Table retrieval

The most complex table in the paper:

In [29]:
# We can retrieve this table
retriever.get_relevant_documents(
    "What do you see on the images in the database?"
)

C:\Users\Admin\AppData\Local\Temp\ipykernel_25992\144532207.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents(


['Conversation Detail description Complex reasoning All Full data 83.1 75.3 96.5 85.1 Detail + Complex 81.5 (-1.6) 73.3 (-2.0) 90.8 (-5.7) 81.9 (-3.2) Conv + 5% Detail + 10% Complex 81.0 (-2.1) 68.4 (-7.1) 91.5 (-5.0) 80.5 (-4.4) Conversation 76.5 (-6.6) 59.8 (-16.2 ) 84.9 (-12.4 ) 73.8 (-11.3 ) No Instruction Tuning 22.0 ( -61.1 ) 24.0 ( -51.3 ) 18.5 ( -78.0 ) 21.5 ( -63.6 )',
 'Visual features Best variant Predict answer ﬁrst Training from scratch 7B model size Before 90.92 - 85.81 (-5.11) 89.84 (-1.08) Last 89.96 (-0.96) 89.77 (-1.15) - -']

We can retrieve this image summary:

In [ ]:
from aift.multimodal import textqa
from aift import setting

# Set Pathumma API Key
setting.set_api_key('API_KEY')

# Define a simple wrapper for Pathumma
class PathummaModel:
    def __init__(self):
        pass

    def generate(self, instruction: str, return_json: bool = False):
        response = textqa.generate(instruction=instruction, return_json=return_json)
        if return_json:
            return response.get("content", "")
        return response

    def __call__(self, input: str):
        return self.generate(input, return_json=False)

# Initialize Pathumma Model
model_local = PathummaModel()

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

after_rag_template = """ตอบคำถามโดยพิจารณาจากบริบทต่อไปนี้เท่านั้น:
{context}
คำถาม: {question}
"""
prompt = ChatPromptTemplate.from_template(after_rag_template)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model_local
    | StrOutputParser()
)

In [33]:
chain.invoke(
     "What do you see on the images in the database?"
)

'On the images in the database, you can see various visual features that are used to train a model. The best variant of these features is used to predict an answer first, and then training from scratch with a 7B model size before achieving certain performance metrics.'